In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar100
from sklearn.model_selection import train_test_split

(x_train, _), (x_test, _) = cifar100.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train, x_test = train_test_split(x_train, test_size=0.1, random_state=42)


input_img = layers.Input(shape=(32, 32, 3))


encoder = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
encoder = layers.MaxPooling2D((2, 2), padding='same')(encoder)
encoder = layers.BatchNormalization()(encoder)
encoder = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(encoder)
encoder = layers.MaxPooling2D((2, 2), padding='same')(encoder)
encoder = layers.BatchNormalization()(encoder)


decoder = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(encoder)
decoder = layers.UpSampling2D((2, 2))(decoder)
decoder = layers.BatchNormalization()(decoder)
decoder = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(decoder)
decoder = layers.UpSampling2D((2, 2))(decoder)
decoder = layers.BatchNormalization()(decoder)
decoder = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(decoder)


autoencoder = models.Model(input_img, decoder)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

autoencoder.summary()


history = autoencoder.fit(x_train, x_train,
                epochs=25,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))